In [31]:
import pandas as pd
import re
import requests
import bs4 as bs
import dateparser
import dateparser

In [2]:
points_table = pd.DataFrame()
for round in range(1,18):
  round_points_table  =pd.read_html(f'https://www.footywire.com/afl/footy/dream_team_round?year=2020&round={round}&p=&s=T')[9]

  round_points_table['Round'] = f'Round {round}'
  points_table = points_table.append(round_points_table)

points_table.columns = points_table.iloc[0]
points_table = points_table.rename({'Round 1':'Round'},axis=1)
points_table = points_table[['Player', 'Team',"2020 R1Score",'Round']]
points_table

,Player,Team,2020 R1Score,Round
0,Player,Team,2020 R1Score,Round 1
1,Jack Viney Injured,Demons,132,Round 1
2,Dylan Shiel,Bombers,122,Round 1
3,Ben Cunnington,Kangaroos,117,Round 1
4,Luke Shuey Injured,Eagles,116,Round 1
...,...,...,...,...
392,Brandon Zerk-Thatcher,Bombers,12,Round 17
393,Sam Docherty,Blues,11,Round 17
394,Toby McLean Injured,Bulldogs,10,Round 17
395,Lewis Taylor,Swans,0,Round 17


In [3]:
replace_team_names =   {"Adelaide":"Crows",   "Brisbane":"Lions","Carlton":"Blues",   "Collingwood":"Magpies",   "Essendon":"Bombers",   "Fremantle":"Dockers",   "GWS":"Giants",   "Geelong":"Cats",   "Gold Coast":"Suns",   "Hawthorn":"Hawks",   "Melbourne":"Demons",   "North Melbourne":"Kangaroos",   "Port Adelaide":"Power",   "Richmond":"Tigers",   "St Kilda":"Saints",   "Sydney":"Swans",   "West Coast":"Eagles",   "Western Bulldogs":"Bulldogs"}

In [29]:
def getRounds(r=2020):
  rounds = pd.read_html(f'https://www.footywire.com/afl/footy/ft_match_list?year={r}')[8]
  rounds['Round_Num'] = rounds [0].str.extract(r'(Round \d{1,2})')
  rounds['Round_Num'] = rounds['Round_Num'].ffill()
  rounds['Team 1'] = rounds[1].str.split(' v ',expand=True)[0]
  rounds['Team 2'] = rounds[1].str.split(' v ',expand=True)[1]
  rounds = rounds.dropna(subset=['Team 2'])
  rounds.columns = rounds.iloc[0]
  rounds = rounds[rounds['Home'] != 'Home']
  rounds = rounds[['Round 1',	'Home',	'Away Teams', 'Date']]
  rounds = rounds.replace(replace_team_names)
  home_teams = rounds['Home'].unique()
  for t in points_table['Team']:
    if t not in home_teams:
      print(t)
      print('are the team mismatches')
  rounds = rounds.append(rounds.rename({'Home': 'Away Teams', 'Away Teams':'Home'},axis=1))
  rounds = rounds.reset_index(drop=True).rename({'Home': 'Team 1', 'Away Teams':'Team 2'},axis=1)
  rounds = rounds.rename({'Round 1': 'Round'},axis=1)
  rounds['Round'] = rounds['Round'].str.slice(start=6)
  rounds['Date'] = rounds['Date'].apply(
    lambda x: dateparser.parse(x)
    )
  return rounds
rounds = getRounds(2020)
nextRounds = getRounds(2021)
nextRounds.to_csv('2021_schedule')

Team
are the team mismatches
Team
are the team mismatches
Team
are the team mismatches
Team
are the team mismatches
Team
are the team mismatches
Team
are the team mismatches
Team
are the team mismatches
Team
are the team mismatches
Team
are the team mismatches
Team
are the team mismatches
Team
are the team mismatches
Team
are the team mismatches
Team
are the team mismatches
Team
are the team mismatches
Team
are the team mismatches
Team
are the team mismatches
Team
are the team mismatches
Team
are the team mismatches
Team
are the team mismatches
Team
are the team mismatches
Team
are the team mismatches
Team
are the team mismatches
Team
are the team mismatches
Team
are the team mismatches
Team
are the team mismatches
Team
are the team mismatches
Team
are the team mismatches
Team
are the team mismatches
Team
are the team mismatches
Team
are the team mismatches
Team
are the team mismatches
Team
are the team mismatches
Team
are the team mismatches
Team
are the team mismatches


In [30]:
nextRounds

1,Round,Team 1,Team 2,Date
0,1,Tigers,Blues,2021-03-18 19:25:00
1,1,Magpies,Bulldogs,2021-03-19 19:50:00
2,1,Demons,Dockers,2021-03-20 13:45:00
3,1,Crows,Cats,2021-03-20 16:05:00
4,1,Lions,Swans,2021-03-20 18:45:00
...,...,...,...,...
391,23,Hawks,Tigers,2021-08-21 12:00:00
392,23,Dockers,Saints,2021-08-21 12:00:00
393,23,Suns,Swans,2021-08-21 12:00:00
394,23,Power,Bulldogs,2021-08-21 12:00:00


In [6]:
Players = pd.DataFrame()
position = 'RU'
for position in ['RU','DE','FO','MI']:
  f = requests.get(f"https://www.footywire.com/afl/footy/dream_team_season?p={position}").text
  soup = bs.BeautifulSoup(f, 'lxml')
  table = soup.find_all('table')[8] 
  parsed_table = soup.find_all('table')[8] 
  data = [[td.a['href'] if td.find('a') else 
            ''.join(td.stripped_strings)
            for td in row.find_all('td')]
          for row in parsed_table.find_all('tr')]
  data2 = [[''.join(td.stripped_strings)
            for td in row.find_all('td')]
          for row in parsed_table.find_all('tr')]
  df = pd.DataFrame(data[1:], columns=data[0])
  df2 = pd.DataFrame(data2[1:], columns=data2[0])
  player_info = df2.join(df['Player'],rsuffix='_url')
  player_info['Position'] = position
  Players = Players.append(player_info)
Players

,Rank,Player,Team,Games,Price,TotalScore,AverageScore,*Value,Player_url,Position
0,1,Matthew Flynn,Giants,1,"$219,000",99,99.0,45.2,pr-greater-western-sydney-giants--matthew-flynn,RU
1,2,Tom Hickey,Swans,1,"$442,000",92,92.0,20.8,pr-sydney-swans--tom-hickey,RU
2,3,Todd Goldstein,Kangaroos,1,"$754,000",90,90.0,11.9,pr-kangaroos--todd-goldstein,RU
3,4,Scott Lycett,Power,1,"$633,000",88,88.0,13.9,pr-port-adelaide-power--scott-lycett,RU
4,4,Ben McEvoy,Hawks,1,"$493,000",88,88.0,17.8,pr-hawthorn-hawks--ben-mcevoy,RU
...,...,...,...,...,...,...,...,...,...,...
144,145,Lachlan McNeil,Bulldogs,1,"$172,000",26,26.0,15.1,pr-western-bulldogs--lachlan-mcneil,MI
145,146,Charlie Constable,Cats,1,"$460,000",25,25.0,5.4,pr-geelong-cats--charlie-constable,MI
146,147,Josh Daicos,Magpies,1,"$621,000",19,19.0,3.1,pr-collingwood-magpies--josh-daicos,MI
147,148,Matt RowellI,Suns,1,"$536,000",16,16.0,3.0,pr-gold-coast-suns--matt-rowell,MI


In [7]:
def getPlayerScores(pu):
  pu = "https://www.footywire.com/afl/footy/"+pu
  fantasy_points = pd.read_html(pu)
  playerTable = pd.DataFrame()
  for n,g in enumerate(fantasy_points):
    if "AFL Fantasy Stats" in str(g.iloc[0,0]):
      tempplayerTable = fantasy_points[n+1]
      tempplayerTable['Year'] = g.iloc[0,0]
      playerTable = playerTable.append(tempplayerTable)
  playerTable = playerTable.rename({0:"Round",2:'Score'},axis=1)[['Round','Year','Score']]
  playerTable = playerTable[playerTable['Round'].str.len()<=2]
  playerTable['Year'] = playerTable['Year'].str.slice(stop=4)
  return playerTable

In [8]:
full_player_scores = pd.DataFrame()
for index, row in Players.iterrows():
  playerScore = getPlayerScores(row['Player_url'])
  playerScore['Name'] = row['Player']
  playerScore['Team'] = row['Team']
  playerScore['Position'] = row['Position']
  playerScore['Price'] = int(row['Price'].replace(',', '').replace('$', ''))
  full_player_scores = full_player_scores.append(rounds.merge(playerScore,left_on=['Team 1','Round'], right_on=['Team', 'Round']))
full_player_scores
full_player_scores.to_csv('full_players.csv')

KeyboardInterrupt: 